# Model Test
Let's see how our model performs on the stage 1 evaluation set.

## Imports
`pneumonia.model_test` has all the necessary functions.

In [ ]:
import sys
sys.path.append("..")
from pneumonia.model_test import *

## Load the label map
This simply maps a class id to a human readable name, in our case 1 => pneumonia

In [ ]:
LABEL_MAP = 'pneumonia/data/pneumonia_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP, use_display_name=True)

## Load the model

In [ ]:
FROZEN_GRAPH = 'pneumonia/models/ssd_mobilenet_v2/exported/frozen_inference_graph.pb'
frozen_graph = load_tf_graph(FROZEN_GRAPH)

## Gather the test images

In [ ]:
TEST_DIR = 'pneumonia/data/stage_1_test_images'
test_images = glob.glob(os.path.join(TEST_DIR, '*.dcm'))

## Run the test
Detect pneumonia opacities from the test images. If detected, show the images with the bounding boxes.

In [ ]:
%matplotlib inline
from utils import visualization_utils as vis_util
from matplotlib import pyplot as plt
for image in test_images:
  # for image in test_images:
    dcm = pydicom.read_file(image)
    image_np = load_dcm_into_numpy_array(dcm)
    output = run_inference_for_single_image(image_np, frozen_graph)
    if output['num_detections'] > 0:
      vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output['detection_boxes'],
        output['detection_classes'],
        output['detection_scores'],
        category_index,
        instance_masks=output.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.1
      )
      plt.figure(figsize=(12, 8))
      plt.imshow(image_np)